In [0]:
from __future__ import print_function, division
import os
from planet4 import io
from os.path import join as pjoin
HOME = os.environ['HOME']

In [0]:
dbname = io.get_current_database_fname()

In [0]:
obsid='ESP_011544_0985'
res = io.ResultManager(obsid)

In [0]:
res.load_dataframes()

In [0]:
res.blotches.columns

In [0]:
res.blotches

In [0]:
imgdata = pd.read_hdf(dbname, 'df',
                      where='image_name='+obsid)#, columns=['image_id', 'x_tile', 'y_tile'])

In [0]:
imgdata

I want the x and y coords of the `res` object because that's the coords of clustering results, but need the x_tile/y_tile from df for the conversion to HiRISE pixels.

In [0]:
p4id = markings.ImageID('APF00006ox')

In [0]:
p4id.plot_all()

In [0]:
from skimage.io import imread
img = imread('/Users/klay6683/data/planet4/images/ESP_011296_0975_RGB.NOMAP.tiff')

In [0]:
df.y_tile.max()

In [0]:
img.shape

In [0]:
p4id.subframe.shape

In [0]:
fig, ax = plt.subplots(nrows=2)
ax[0].imshow(p4id.subframe)
ax[1].imshow(img[:648])

In [0]:
df.columns

In [0]:
newimg = np.zeros((5, 82))

In [0]:
for index,data in zip(indices, imgdata):
    newimg[index[0], index[1]]= data

In [0]:
plt.figure()
plt.imshow(newimg, aspect='equal')

In [0]:
for _, blotch in res.blotches.iterrows():
    for _, fan in res.fans.iterrows():
        if abs(blotch.x - fan.x) < 10 and abs(blotch.y - fan.y) < 10:
            print("inside 10")
            print("Blotch", blotch, "Fan", fan)

In [0]:
%matplotlib qt

In [0]:
def plot_image_id(i=0):
    imgid = res.blotches.image_id.unique().tolist()[i]
    
    # image 1
    p4 = markings.ImageID(imgid)
    fig = p4.show_subframe(aspect='equal')
    fig.get_axes()[0].set_title("Planet4 id {}".format(imgid))
    fig.savefig('plots/'+imgid+'_1.png', dpi=150)
    plt.close(fig)
    
    # image 2
    fig, ax = plt.subplots(figsize=markings.calc_4_3(8))
    p4.plot_fans(ax=ax)
    p4.plot_blotches(ax=ax)
    ax.set_title('Planet4 id {}, Original markings'.format(imgid))
    fig.savefig('plots/'+imgid+'_2.png', dpi=150)
    plt.close(fig)
    
    # image 3
    reduced_blotches = res.blotches[res.blotches.image_id==imgid]
    reduced_fans = res.fans[res.fans.image_id==imgid]
    fig, ax = plt.subplots(figsize=markings.calc_4_3(8))
    p4.plot_fans(ax=ax, fans=reduced_fans)
    p4.plot_blotches(ax=ax, blotches=reduced_blotches)
    ax.set_title('Planet4 id {}, Clustered markings'.format(imgid))
    fig.savefig('plots/'+imgid+'_3.png', dpi=150)
    plt.close(fig)
#     plt.show()

In [0]:
length = len(res.blotches.image_id.unique())
for i in range(length):
# for i in range(3):
    print(i*100/length,'%')
    plot_image_id(i)

In [0]:
from IPython.html.widgets import interact

In [0]:
interact(plot_image_id, i=(0, len(res.blotches.image_id.unique().tolist())-1))

In [0]:
blotches = p4data[p4data.marking=='blotch']